In [23]:
import pandas as pd
print(pd.__version__)

# Load the uploaded file
directory="/your_directory/"
file_path = str(directory + "Paralogs_genes_70_coordinates_v2.tsv")
df = pd.read_csv(file_path, sep="\t")

# Display the first few rows to understand its structure
df.head()


1.3.5


,Gene.symbol,Gene.secondaryIdentifier,Gene.chromosome.primaryIdentifier,Gene.chromosomeLocation.start,Gene.chromosomeLocation.end
0,AAC3,YBR085W,chrII,415983,416906
1,AAD10,YJR155W,chrX,727405,728271
2,AAD14,YNL331C,chrXIV,16118,17248
3,AAD15,YOL165C,chrXV,1647,2078
4,AAD3,YCR107W,chrIII,313890,314981


1- Specify the resection tract length in bp and compute the genomic intervals a DSB can fall into to expose a paralog gene.
2- Compute the sum of intervals. Remove overlapping regions between interval so that they do not count twice.
3- Export the merged intervals into a bedgraph format
4- print total interval length

In [24]:

# Define the resection tract length in bp
expansion = 16000

# Function to expand genomic interval
def expand_interval(row, expansion):
    start = row[3]
    end = row[4]
    # Determine strand based on start and end
    if start < end:
        strand = 1
        new_start = max(0, start - expansion)
        new_end = end + expansion
    else:
        strand = -1
        new_start = max(0, end - expansion)
        new_end = start + expansion
    return pd.Series([new_start, new_end, strand])

# Apply the function to each row
df[['Expanded_Start', 'Expanded_End', 'Strand']] = df.apply(expand_interval, axis=1, expansion=expansion)

# Display the updated DataFrame
df.head()

# First, create a DataFrame with only the relevant columns for interval merging
intervals = df[['Gene.chromosome.primaryIdentifier', 'Expanded_Start', 'Expanded_End']].copy()
intervals.columns = ['Chromosome', 'Start', 'End']

# Convert start and end to integers (safety check)
intervals['Start'] = intervals['Start'].astype(int)
intervals['End'] = intervals['End'].astype(int)

# Function to merge overlapping intervals
def merge_intervals(intervals_df):
    merged_intervals = []
    for chrom in intervals_df['Chromosome'].unique():
        chrom_df = intervals_df[intervals_df['Chromosome'] == chrom].sort_values(by='Start')
        current_start, current_end = None, None
        for _, row in chrom_df.iterrows():
            start, end = row['Start'], row['End']
            if current_start is None:
                current_start, current_end = start, end
            elif start <= current_end:  # overlapping
                current_end = max(current_end, end)
            else:
                merged_intervals.append((chrom, current_start, current_end))
                current_start, current_end = start, end
        if current_start is not None:
            merged_intervals.append((chrom, current_start, current_end))
    return pd.DataFrame(merged_intervals, columns=['Chromosome', 'Start', 'End'])

# Merge overlapping intervals
merged_intervals_df = merge_intervals(intervals)

# Calculate the total length of merged intervals
merged_intervals_df['Length'] = merged_intervals_df['End'] - merged_intervals_df['Start']
total_length = merged_intervals_df['Length'].sum()

merged_intervals_df.head(), total_length

# Prepare the merged intervals in BEDGraph format
# BEDGraph format: chrom, start, end, value
# We'll set value as 1 to indicate presence of an expanded gene region

bedgraph_df = merged_intervals_df[['Chromosome', 'Start', 'End']].copy()
bedgraph_df['Value'] = 1

# Save to a .bedgraph file
bedgraph_df.to_csv(str(directory + "Paralogs_merged_resection_" + str(expansion) + "bp.bedgraph"), sep='\t', header=False, index=False)

print("Total DSB space at risk of paralog-mediated rearrangement is " + str(total_length) + " bp")
print("equal to %.2f" % float(total_length/12100000*100) + "% of the genome" )

Total DSB space at risk of paralog-mediated rearrangement is 5179102 bp
equal to 42.80% of the genome
